In [3]:
from Bio.Blast import NCBIXML
import pandas as pd
from pandas.io.json import json_normalize


from Bio.Blast import NCBIWWW
from datetime import datetime
startTime = datetime.now()

with open("sample.fasta", "r") as fasta_file:
    sequences = fasta_file.read()
    fasta_file.close()

    result_handle = NCBIWWW.qblast("blastn", "nt", sequences)
    save_result = open("blast_result.xml", "w")
    save_result.write(result_handle.read())
    save_result.close()
    result_handle.close()


result = open("blast_result.xml", "r")
records = NCBIXML.parse(result)

#Create the outer list
final = []
for blast_record in records:
    dic = {}
    #print(blast_record.query)
    query_title = blast_record.query.split("|")[0]
    dic["id"] = query_title
    pairs = []
    for i in blast_record.descriptions:
        ncbi = i.title.split("|")[3]
        name = " ".join(i.title.split("|")[4].split(" ")[1:3])
        e_value = i.e
        score = i.score
        dic_pairs = {}
        # Make a dictionary containing the name and the e_value
        dic_pairs["hit"] = name
        dic_pairs["e-value"] = e_value
        dic_pairs["ncbi"] = ncbi
        dic_pairs["score"] = score
        dic_pairs["gi_number"] = i.title.split("|")[1]
        pairs.append(dic_pairs)
        #Maybe use a counter to break out of the loop
    k =[]
    for alignment in blast_record.alignments:
       # gi_alignment = alignment.title.split("|")[1]
        #identity["id2"] = gi_alignment
        new = {}
        #print(alignment.hsps.align_length)
        for hsp in alignment.hsps:
            new["coverage"] = hsp.align_length / blast_record.query_length
            new["bits"] = hsp.bits
            new["percent"] = hsp.identities / hsp.align_length
            k.append(new)

    dic["pairs"] = pairs
    dic["k"] = k
    final.append(dic)


In [4]:
ndata_pairs = json_normalize(final, 'pairs', ['id'])
ndata_k = json_normalize(final, 'k', ['id'])
df_pairs=pd.DataFrame(ndata_pairs)
df_k = pd.DataFrame(ndata_k)

In [5]:
fainal_df = pd.merge(df_k, df_pairs, left_index=True, right_index=True)

In [6]:
fainal_df.head()

,bits,coverage,percent,id_x,e-value,gi_number,hit,ncbi,score,id_y
0,1020.190,1.000000,1.000000,species1,0.000000e+00,1377213559,Raffaelea cyclorhipidia,NR_155726.1,1130.0,species1
1,1020.190,1.000000,1.000000,species1,0.000000e+00,1098884338,Raffaelea cyclorhipidia,KX267136.1,1130.0,species1
2,600.005,0.959292,0.845018,species1,2.452900e-167,294440760,Raffaelea canadensis,GQ225699.1,664.0,species1
3,538.690,0.750442,0.893868,species1,7.033880e-149,107857293,Sporothrix cf.,DQ539556.1,596.0,species1
4,538.690,0.750442,0.893868,species1,7.033880e-149,107857292,Sporothrix cf.,DQ539555.1,596.0,species1
